# Hands-On with Generative AI Assignment

Here you will gain hands-on experience working with GPT language models via the OpenAI API. Specifically, you will:
1. Create an OpenAI account and obtain an API key.
2. Try generating outputs from several different models available via the API.
3. Gain a first exposure to prompt engineering by experimenting with using multiple prompts for a given task.

---
## Problem 1

Create an [OpenAI account](https://openai.com/index/openai-api), then generate an API key.

## Solution

After logging into your account, go to [https://platform.openai.com/api-keys](https://platform.openai.com/api-keys). Click the "Create new secret key" button to create a new API key. Copy your newly created API key upon creation.

---
## Problem 2

Use the OpenAI API to produce responses to a given prompt using two different models. First use the GPT-3.5 Turbo model to evaluate the prompt, then use the GPT-4 Turbo model to evaluate the same prompt.

For your prompt, try asking the models to perform the following task which requires a bit of reasoning to solve:
```
Andrew is free from 11 am to 3 pm, Joanne is free from noon to 2 pm and then 3:30 pm to 5 pm.
Hannah is available at noon for half an hour, and then 4 pm to 6 pm.
What are some options for start times for a 30 minute meeting for Andrew, Hannah, and Joanne?
```
Compare the quality of the outputs that are obtained from the two different models.

## Solution

To complete this problem we will need to perform the following steps:
1. Install the `openai` Python package.
2. Create an OpenAI client using your API key. In these solutions we load our key from Google drive to avoid hard-coding the API key in our solution.
3. Pass the prompt above to the `chat.completions` API using the `gpt-3.5-turbo` model.
4. Pass the prompt above to the `chat.completions` API using the `gpt-4-turbo` model.
5. Compare the outputs obtained by both models.

In [1]:
# Install the openai package
!pip install openai

In [2]:
# Import packages that you will use for accessing the OpenAI API
import json
from google.colab import drive
from openai import OpenAI

In [3]:
# Mount Google Drive.
# We will get our OpenAI API key from a file that we stored in Google Drive.
drive.mount("/content/gdrive")

Mounted at /content/gdrive


In [4]:
# Read in API key
with open("/content/gdrive/MyDrive/keys.json", "r") as f:
  api_key = json.loads(f.read())["api_key"]

In [5]:
# Create an OpenAI client
client = OpenAI(api_key=api_key)

In [6]:
# Generating response from gpt-3.5-turbo

prompt = """
Andrew is free from 11 am to 3 pm, Joanne is free from noon to 2 pm and then 3:30 pm to 5 pm.
Hannah is available at noon for half an hour, and then 4 pm to 6 pm.
What are some options for start times for a 30 minute meeting for Andrew, Hannah, and Joanne?
"""

# Note the use of the "temperature" parameter...
# ...this allows us to generate reproducable outputs
openai_response = client.chat.completions.create(
    model = 'gpt-3.5-turbo',
    messages = [{'role': 'user', 'content': prompt}],
    temperature = 0
)

# Print the response
print(openai_response.choices[0].message.content)

Some options for start times for a 30-minute meeting for Andrew, Hannah, and Joanne could be:
- 12:00 pm
- 12:30 pm
- 4:00 pm


In [7]:
# Now generating response from gpt-4-turbo
openai_response = client.chat.completions.create(
    model = 'gpt-4-turbo',
    messages = [{'role': 'user', 'content': prompt}],
    temperature = 0
)

# Print the response
print(openai_response.choices[0].message.content)

PermissionDeniedError: Error code: 403 - {'error': {'message': 'Project `proj_6amWdXZiZG5XHtR3u182LRLv` does not have access to model `gpt-4-turbo`', 'type': 'invalid_request_error', 'param': None, 'code': 'model_not_found'}}

**Comparison:** GPT-4 produced a higher quality output in this case. GPT-3.5 incorrectly states that 3:30 pm - 4:00 pm and 4:00 pm - 4:30 pm are options for the meeting. GPT-4 correctly reasons that 12-12:30 is the only feasible time.

---
## Problem 3
Here you will see an example of how the prompting strategy can impact the quality of the output produced. In this problem you will again apply GPT-3.5 to the reasoning task from Problem 2. However, unlike in Problem 2, modify your prompt to include some general step-by-step instructions for solving a scheduling task like the one given. Can you produce a prompt that enables GPT-3.5 to arrive at a correct answer?

## Solution

We will modify our prompt by including the following general step-by-step instructions for solving a scheduling task:
```
You will be given a complex scheduling task below. Solve this task step-by-step by:
- First identifying all time slots that each individual is available.
- Then finding all time slots that all individuals are available.
- Then selecting a time slot from among those where all individuals are available.
```
Note that these instructions are fairly generic, and don't provide any hints specific to the details of the given scheduling problem. However, this additional instruction appears sufficient to guide GPT-3.5 to a correct answer.


In [ ]:
# Generating response from gpt-3.5-turbo

prompt = """
You will be given a complex scheduling task below. Solve this task step-by-step by:
- First identifying all time slots that each individual is available.
- Then finding all time slots that all individuals are available.
- Then selecting a time slot from among those where all individuals are available.

Task: Andrew is free from 11 am to 3 pm, Joanne is free from noon to 2 pm and then 3:30 pm to 5 pm.
Hannah is available at noon for half an hour, and then 4 pm to 6 pm.
What are some options for start times for a 30 minute meeting for Andrew, Hannah, and Joanne?
"""

openai_response = client.chat.completions.create(
    model = 'gpt-3.5-turbo',
    messages = [{'role': 'user', 'content': prompt}],
    temperature = 0
)

# Print the response
print(openai_response.choices[0].message.content)

First, let's identify all the time slots that each individual is available:

Andrew: 11 am - 3 pm
Joanne: 12 pm - 2 pm, 3:30 pm - 5 pm
Hannah: 12 pm - 12:30 pm, 4 pm - 6 pm

Next, let's find all time slots where all individuals are available:
- The only overlapping time slot where all three individuals are available is from 12 pm to 12:30 pm.

Therefore, the best option for a 30-minute meeting for Andrew, Hannah, and Joanne would be to schedule it at 12 pm.
